# Machine Learning

This is a problem of classification where what I am looking for is to identifying whether the employee is living the company or not.

In [30]:
import pandas as pd
import numpy as np

In [31]:
df_cleaned=pd.read_csv('data/HR_data_cleaned.csv')

In [32]:
df_cleaned.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,high,low,medium,department_numeric,left
0,0.38,0.53,2,157,3,0,0,0,1,0,0,1
1,0.80,0.86,5,262,6,0,0,0,0,1,0,1
2,0.11,0.88,7,272,4,0,0,0,0,1,0,1
3,0.72,0.87,5,223,5,0,0,0,1,0,0,1
4,0.37,0.52,2,159,3,0,0,0,1,0,0,1


In [33]:
df_cleaned.shape

(11991, 12)

In [34]:
df_cleaned.columns

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'high', 'low', 'medium', 'department_numeric',
       'left'],
      dtype='object')

In [35]:
x=df_cleaned[['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'high', 'low', 'medium', 'department_numeric']] #X will be the features
y=df_cleaned['left'] #left column will be the target one (1=left, 0 =stay)

In [36]:
#Split the dataset into two parts: train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [37]:
#checking we have the desired 70% train, 30% split of the data
print("{0:0.2f}% in training set".format((len(x_train)/len(df_cleaned.index)) * 100))
print("{0:0.2f}% in test set".format((len(x_test)/len(df_cleaned.index)) * 100))

66.99% in training set
33.01% in test set


In [38]:
### Preprocesado de variables

## Choosing the model

### Looking for the best accuracy, I will analyse some prediction models 

### Let's start with _K-nearest Neighbors_

In [39]:
from sklearn.neighbors import KNeighborsClassifier
clf1 = KNeighborsClassifier(n_neighbors=10)

In [40]:
clf1.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=10, p=2,
           weights='uniform')

In [41]:
from sklearn.metrics import accuracy_score

In [42]:
accuracy_score(y_test,clf1.predict(x_test))

0.9295098534613441

In [43]:
from sklearn.model_selection import GridSearchCV

In [44]:
clf1=GridSearchCV(estimator =KNeighborsClassifier(n_neighbors=5),
                  param_grid={"n_neighbors":np.arange(1,50)})

clf1.fit(x,y) #training the model with the best_params
clf1.best_params_

{'n_neighbors': 4}

In [45]:
from sklearn.model_selection import cross_val_score

In [46]:
cross_val_score(KNeighborsClassifier(n_neighbors=4),x,y).mean()

0.9298633941450096